In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:

file_path = 'SubscriptionUseCase_Dataset.xlsx'


user_df = pd.read_excel(file_path, sheet_name='User_Data')
subs_df = pd.read_excel(file_path, sheet_name='Subscriptions')
logs_df = pd.read_excel(file_path, sheet_name='Subscription_Logs')
plans_df = pd.read_excel(file_path, sheet_name='Subscription_Plans')
billing_df = pd.read_excel(file_path, sheet_name='Billing_Information')


In [ ]:

print("Users:")
display(user_df.head())
print("Subscriptions:")
display(subs_df.head())
print("Logs:")
display(logs_df.head())
print("Plans:")
display(plans_df.head())
print("Billing Information:")
display(billing_df.head())


Users:


,User Id,Name,Phone,Email,Status
0,1,User1,1234567801,user1@example.com,active
1,2,User2,1234567802,user2@example.com,inactive
2,3,User3,1234567803,user3@example.com,inactive
3,4,User4,1234567804,user4@example.com,active
4,5,User5,1234567805,user5@example.com,inactive


Subscriptions:


,Subscription Id,Subscription Type,Product Id,User Id,Status,Start Date,Last Billed Date,Last Renewed Date,Terminated Date,Grace Time
0,1,monthly,46,61,PAUSED,2024-04-20,2024-01-20,2024-09-06,NaN,5
1,2,monthly,35,37,active,2024-08-24,2024-03-19,2024-10-29,NaN,5
2,3,monthly,96,24,active,2024-01-26,2023-12-10,2024-09-04,NaN,5
3,4,monthly,79,69,active,2024-07-31,2024-11-21,2024-07-30,NaN,5
4,5,monthly,52,3,active,2024-07-23,2024-04-05,2024-10-08,NaN,5


Logs:


,Subscription id,current status,next status,action,action date
0,30,initialized,paused,renew,2024-05-17
1,51,active,paused,billing_success,2024-10-28
2,15,active,paused,renew,2024-09-18
3,69,active,paused,renew_failed,2024-08-09
4,99,initialized,active,renew_failed,2024-07-23


Plans:


,Product Id,Name,Price,Auto Renewal Allowed,Status
0,1,Plan1,57.65,No,Active
1,2,Plan2,15.30,Yes,Active
2,3,Plan3,73.86,Yes,Active
3,4,Plan4,27.82,Yes,Active
4,5,Plan5,42.58,Yes,Active


Billing Information:


,billing_id,subscription_id,amount,billing_date,payment_status,Unnamed: 5,Unnamed: 6
0,1,54,308.56,2024-01-04,paid,NaN,NaN
1,2,52,200.11,2024-05-26,paid,NaN,NaN
2,3,6,407.49,2024-06-24,paid,NaN,NaN
3,4,100,289.68,2024-04-09,pending,NaN,NaN
4,5,16,114.07,2024-11-22,pending,NaN,NaN


In [5]:
print(user_df.info(), '\n')
print(subs_df.info(), '\n')
print(logs_df.info(), '\n')
print(plans_df.info(), '\n')
print(billing_df.info(), '\n')

print("Missing values in Users:\n", user_df.isnull().sum())
print("Missing values in Subscriptions:\n", subs_df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   User Id  100 non-null    int64 
 1   Name     100 non-null    object
 2   Phone    100 non-null    int64 
 3   Email    100 non-null    object
 4   Status   100 non-null    object
dtypes: int64(2), object(3)
memory usage: 4.0+ KB
None 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Subscription Id    100 non-null    int64  
 1   Subscription Type  100 non-null    object 
 2   Product Id         100 non-null    int64  
 3   User Id            100 non-null    int64  
 4   Status             100 non-null    object 
 5   Start Date         100 non-null    object 
 6   Last Billed Date   100 non-null    object 
 7   Last Renewed Date  100 non-null    objec

In [8]:
# Fill missing values in object (string) columns with '',
# and in numeric columns with 0 (customize for your needs)

def safe_fillna(df):
    # Fill object columns with ''
    obj_cols = df.select_dtypes(include=['object']).columns
    df[obj_cols] = df[obj_cols].fillna('')
    # Fill number columns with 0
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].fillna(0)
    return df

user_df = safe_fillna(user_df)
subs_df = safe_fillna(subs_df)
logs_df = safe_fillna(logs_df)
plans_df = safe_fillna(plans_df)
billing_df = safe_fillna(billing_df)

# Convert date columns to datetime if present
date_cols = ['Start Date', 'Last Billed Date', 'Last Renewed Date', 'Terminated Date']
for col in date_cols:
    if col in subs_df.columns:
        subs_df[col] = pd.to_datetime(subs_df[col], errors='coerce')

if 'billing_date' in billing_df.columns:
    billing_df['billing_date'] = pd.to_datetime(billing_df['billing_date'], errors='coerce')


In [ ]:

merged_df = pd.merge(subs_df, user_df, on='User Id', suffixes=('_subscription', '_user'))
merged_df = pd.merge(merged_df, plans_df, left_on='Product Id', right_on='Product Id', suffixes=('', '_plan'))

display(merged_df.head())


,Subscription Id,Subscription Type,Product Id,User Id,Status_subscription,Start Date,Last Billed Date,Last Renewed Date,Terminated Date,Grace Time,Name,Phone,Email,Status_user,Name_plan,Price,Auto Renewal Allowed,Status
0,1,monthly,46,61,PAUSED,2024-04-20,2024-01-20,2024-09-06,1970-01-01,5,User61,1234567861,user61@example.com,inactive,Plan46,40.45,Yes,Active
1,2,monthly,35,37,active,2024-08-24,2024-03-19,2024-10-29,1970-01-01,5,User37,1234567837,user37@example.com,inactive,Plan35,95.92,No,Active
2,3,monthly,96,24,active,2024-01-26,2023-12-10,2024-09-04,1970-01-01,5,User24,1234567824,user24@example.com,active,Plan96,23.86,Yes,Active
3,4,monthly,79,69,active,2024-07-31,2024-11-21,2024-07-30,1970-01-01,5,User69,1234567869,user69@example.com,active,Plan79,98.68,Yes,Active
4,5,monthly,52,3,active,2024-07-23,2024-04-05,2024-10-08,1970-01-01,5,User3,1234567803,user3@example.com,inactive,Plan52,61.25,Yes,Active


In [10]:
# Remove duplicates if any
merged_df.drop_duplicates(inplace=True)

# Choose columns relevant for modeling (customize as needed)
selected_cols = ['User Id', 'Status_subscription', 'Start Date', 'Last Renewed Date', 'Price', 'Auto Renewal Allowed', 'Status_user']
model_df = merged_df[selected_cols]

display(model_df.head())


,User Id,Status_subscription,Start Date,Last Renewed Date,Price,Auto Renewal Allowed,Status_user
0,61,PAUSED,2024-04-20,2024-09-06,40.45,Yes,inactive
1,37,active,2024-08-24,2024-10-29,95.92,No,inactive
2,24,active,2024-01-26,2024-09-04,23.86,Yes,active
3,69,active,2024-07-31,2024-07-30,98.68,Yes,active
4,3,active,2024-07-23,2024-10-08,61.25,Yes,inactive


In [11]:
# Calculate tenure and activity stats as an example feature
model_df['subscription_tenure_days'] = (pd.to_datetime('today') - model_df['Start Date']).dt.days

# Show feature summary statistics
display(model_df.describe())


C:\Users\KIIT\AppData\Local\Temp\ipykernel_16352\2659240706.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df['subscription_tenure_days'] = (pd.to_datetime('today') - model_df['Start Date']).dt.days


,User Id,Start Date,Last Renewed Date,Price,subscription_tenure_days
count,100.0000,100,100,100.000000,100.000000
mean,47.3400,2024-05-18 05:02:24,2024-06-16 03:21:36,62.963100,482.790000
min,2.0000,2023-12-16 00:00:00,2023-12-03 00:00:00,14.240000,297.000000
25%,20.0000,2024-02-29 12:00:00,2024-03-17 06:00:00,43.587500,409.000000
50%,45.0000,2024-05-07 00:00:00,2024-06-17 00:00:00,68.775000,494.000000
75%,76.0000,2024-07-31 00:00:00,2024-09-23 06:00:00,85.095000,561.500000
max,98.0000,2024-11-20 00:00:00,2024-11-29 00:00:00,98.710000,637.000000
std,29.0931,NaN,NaN,24.759827,98.427771


In [12]:
model_df.to_csv('cleaned_subscription_data.csv', index=False)


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [15]:
df = pd.read_csv('cleaned_subscription_data.csv')

In [16]:
# 3. Pre-Processing: Encode categorical columns as numbers
# (Adjust column names if needed)
df['Auto Renewal Allowed'] = df['Auto Renewal Allowed'].map({'Yes': 1, 'No': 0})
df['Status_user'] = df['Status_user'].map({'active': 1, 'inactive': 0})
df['Status_subscription'] = df['Status_subscription'].map({'active': 0, 'PAUSED': 1, 'paused': 1, 'inactive': 1}) # paused/inactive=1=churn

In [17]:
# 4. Define features (X) and target (y)
feature_cols = ['Price', 'Auto Renewal Allowed', 'Status_user', 'subscription_tenure_days']
X = df[feature_cols]
y = df['Status_subscription']   # 1 = churn, 0 = not churn

In [18]:
# 5. Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# 6. Train Model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [19]:
# 7. Predict & Evaluate
y_pred = clf.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy Score:", accuracy_score(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.47      0.88      0.61         8
           1       0.90      0.53      0.67        17

    accuracy                           0.64        25
   macro avg       0.68      0.70      0.64        25
weighted avg       0.76      0.64      0.65        25

Confusion Matrix:
 [[7 1]
 [8 9]]
Accuracy Score: 0.64


In [20]:
# 8. Predict on ALL records (probabilities for dashboard)
df['churn_prediction'] = clf.predict(X)
df['churn_probability'] = clf.predict_proba(X)[:,1]

In [21]:
# 9. Show Results: User Id, Churn Prediction, Probability
result = df[['User Id', 'churn_prediction', 'churn_probability', 'Price', 'subscription_tenure_days']]
display(result.head(10))

,User Id,churn_prediction,churn_probability,Price,subscription_tenure_days
0,61,1,0.75,40.45,511
1,37,0,0.03,95.92,385
2,24,0,0.20,23.86,596
3,69,0,0.11,98.68,409
4,3,0,0.41,61.25,417
5,86,0,0.23,72.32,490
6,45,1,0.88,57.73,537
7,28,0,0.22,60.80,494
8,97,1,0.85,93.24,529
9,26,1,0.87,74.74,513


In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score
import xgboost as xgb

# Define models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
    "SVM": SVC(probability=True, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "XGBoost": xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
}

results = {}
trained_models = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n=== {name} ===")
    print("Accuracy: ", acc)
    print(classification_report(y_test, y_pred))
    results[name] = acc
    trained_models[name] = model

# Find the best model by accuracy
best_model_name = max(results, key=results.get)
best_model = trained_models[best_model_name]

print(f"\nThe best model is: {best_model_name} (Accuracy: {results[best_model_name]:.4f})")



=== Random Forest ===
Accuracy:  0.64
              precision    recall  f1-score   support

           0       0.47      0.88      0.61         8
           1       0.90      0.53      0.67        17

    accuracy                           0.64        25
   macro avg       0.68      0.70      0.64        25
weighted avg       0.76      0.64      0.65        25


=== Logistic Regression ===
Accuracy:  0.76
              precision    recall  f1-score   support

           0       0.60      0.75      0.67         8
           1       0.87      0.76      0.81        17

    accuracy                           0.76        25
   macro avg       0.73      0.76      0.74        25
weighted avg       0.78      0.76      0.77        25


=== SVM ===
Accuracy:  0.84
              precision    recall  f1-score   support

           0       0.70      0.88      0.78         8
           1       0.93      0.82      0.88        17

    accuracy                           0.84        25
   macro avg   

c:\Users\KIIT\Documents\subscription-ml\venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [13:40:28] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



=== XGBoost ===
Accuracy:  0.52
              precision    recall  f1-score   support

           0       0.39      0.88      0.54         8
           1       0.86      0.35      0.50        17

    accuracy                           0.52        25
   macro avg       0.62      0.61      0.52        25
weighted avg       0.71      0.52      0.51        25


The best model is: SVM (Accuracy: 0.8400)


In [32]:
import sys
print(sys.executable)


c:\Users\KIIT\Documents\subscription-ml\venv\Scripts\python.exe


In [41]:
result.to_csv('user_churn_predictions_final.csv', index=False)

In [43]:
import pandas as pd
import json

# Load your CSV
df = pd.read_csv('user_churn_predictions_final.csv')

# Convert DataFrame to a list of dictionaries
json_obj = df.to_dict(orient='records')

# Pretty-print JSON as string with indentation
json_str = json.dumps(json_obj, indent=4)

# Save to file (pretty-printed and readable)
with open('user_churn_predictions.json', 'w') as f:
    f.write(json_str)

# Print (first 500 characters for preview)
print(json_str[:500])


[
    {
        "User Id": 61,
        "churn_prediction": 1,
        "churn_probability": 0.5149246334080501,
        "Price": 40.45,
        "subscription_tenure_days": 511
    },
    {
        "User Id": 37,
        "churn_prediction": 0,
        "churn_probability": 0.4001651339617171,
        "Price": 95.92,
        "subscription_tenure_days": 385
    },
    {
        "User Id": 24,
        "churn_prediction": 1,
        "churn_probability": 0.5666866862365709,
        "Price": 23.86,
     
